# [Puntajes PSU](http://www.t13.cl/noticia/nacional/te-puede-servir/ranking-colegios-mejor-promedio-psu-2016)

In [30]:
from robobrowser import RoboBrowser
import geocoder
import folium
import pandas
import os
from utils import persist_to_file
cwd = os.getcwd()

browser = RoboBrowser(history=True, parser='html5lib')
url = 'http://www.t13.cl/noticia/nacional/te-puede-servir/ranking-colegios-mejor-promedio-psu-2016'
browser.open(url)

ModuleNotFoundError: No module named '__main__.utils'; '__main__' is not a package

In [4]:
table=browser.select('.article-content table')[0]
t=table.__str__()
t[:1000]

'<table border="0" cellpadding="0" cellspacing="0" style="width:1059px;" width="1058">\n\t<colgroup>\n\t\t<col/>\n\t\t<col/>\n\t\t<col/>\n\t\t<col/>\n\t\t<col/>\n\t\t<col/>\n\t</colgroup>\n\t<tbody>\n\t\t<tr height="20">\n\t\t\t<td height="20" style="height: 20px; width: 143px; text-align: center;"><strong>Posición en el ranking</strong></td>\n\t\t\t<td style="width: 159px; text-align: center;"><strong>Tipo de establecimiento</strong></td>\n\t\t\t<td style="width: 131px; text-align: center;"><strong>Comuna\xa0</strong></td>\n\t\t\t<td style="width: 332px; text-align: center;"><strong>Colegio</strong></td>\n\t\t\t<td style="width: 215px; text-align: center;"><strong>Alumnos que rindieron la prueba</strong></td>\n\t\t\t<td style="width: 80px; text-align: center;"><strong>Promedio PSU</strong></td>\n\t\t</tr>\n\t\t<tr height="20">\n\t\t\t<td height="20" style="height: 20px; text-align: center;">1</td>\n\t\t\t<td style="text-align: center;">Particular</td>\n\t\t\t<td style="text-align: cen

In [5]:
t=t.replace(',','.')
df=pandas.read_html(t, header=0)[0]
df.head(10)

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU
0,1,Particular,Providencia,Cambridge College,29,704.4
1,2,Particular,Las Condes,Cordillera,63,699.0
2,3,Particular,Concón,Montemar,22,694.0
3,4,Particular,Maipú,Internacional Alba,26,693.4
4,5,Particular,Vitacura,Los Andes,69,692.9
5,6,Particular,Puerto Montt,Instituto Alemán,46,690.9
6,7,Particular,Lo Barnechea,Maimonides School,10,689.7
7,8,Particular,Chiguayante,Pinares,16,685.5
8,9,Particular,Angol,San José,19,683.7
9,10,Particular,La Reina,The Grange School,133,682.6


In [6]:
g=geocoder.google('colegio the grange school, la reina, chile')
print(g)
print(g.latlng)

<[ERROR - No results found] Google - Geocode [empty]>


ValueError: ERROR - No results found

In [ ]:
g.latlng

[-36.8883627, -73.0351594](
https://www.google.cl/maps/@-33.4382165,-70.5686165,3a,75y,3.98h,101.74t/data=!3m7!1e1!3m5!1sxP13kKF1tvm6pau-DNnsPA!2e0!6s%2F%2Fgeo1.ggpht.com%2Fcbk%3Fpanoid%3DxP13kKF1tvm6pau-DNnsPA%26output%3Dthumbnail%26cb_client%3Dmaps_sv.tactile.gps%26thumb%3D2%26w%3D203%26h%3D100%26yaw%3D33.012394%26pitch%3D0%26thumbfov%3D100!7i13312!8i6656)

In [25]:
from joblib import Memory
import warnings
warnings.filterwarnings('ignore')
memory = Memory(cachedir='colegios', verbose=0)
import time
from memorize.memorize import Memorize

import json

def persist_to_file(file_name):
    def decorator(original_func):

        try:
            cache = json.load(open(file_name, 'r'))
        except (IOError, ValueError):
            cache = {}

        def new_func(param):
            if param not in cache:
                cache[param] = original_func(param)
                json.dump(cache, open(file_name, 'w'))
            return cache[param]

        return new_func

    return decorator

@persist_to_file('cache.dat')
def get_latlng(query):
    providers = geocoder.osm, geocoder.google
    for provider in providers:
        #print(provider)
        try:  
            g=provider(query)
        except ValueError:
            continue
        if g:
            return g.latlng
    return None, None

def get_latlng_nocache(query):
    providers = geocoder.osm, geocoder.google
    for provider in providers:
        #print(provider)
        try:  
            g=provider(query)
        except ValueError:
            continue
        if g:
            return g.latlng
    return None, None

for index,row in df.iterrows():
    stopwords = 'colegio', 'liceo', 'school', 'instituto', 'college'
    query = row.Colegio +", "+ row.Comuna + ", chile"
    query = query.lower()
    for i in stopwords:
        if i in query:
            break
    else:
         query = 'colegio ' + query
    print(query),
    g=get_latlng(query)
    time.sleep(0.1)
    if g ==[None, None]:
        g = get_latlng_nocache(query)
        if not g:
            print('ERROR', query)
            continue
    if g:
        lat, lng = g
        df.set_value(index,'lat', lat)
        df.set_value(index,'lng', lng)
        print(g)
    else:
        print(g, 'ERROR', query)

cambridge college, providencia, chile
[-33.4297301, -70.6171096621219]
colegio cordillera, las condes, chile
[-33.39712965, -70.5143209449632]
colegio montemar, concón, chile
[-32.95104785, -71.5354626703725]
colegio internacional alba, maipú, chile
[-33.4805743, -70.74499]
colegio los andes, vitacura, chile
[-33.37579495, -70.5270086976089]
instituto alemán, puerto montt, chile
[-41.4703149, -72.9327801328877]
maimonides school, lo barnechea, chile
[-33.362362, -70.485016]
colegio pinares, chiguayante, chile
[-36.9329182, -73.0240795401104]
colegio san josé, angol, chile
(None, None)
the grange school, la reina, chile
[-33.43693825, -70.5678364497786]
instituto hebreo, lo barnechea, chile
[-33.36782585, -70.50897645]
colegio trebulco, talagante, chile
[-33.68492175, -70.9181438684788]
colegio tabancura, vitacura, chile
[-33.3803347, -70.5348377]
the kent school, providencia, chile
[-33.442454, -70.606202]
colegio la girouette, las condes, chile
[-33.42014435, -70.5684437802117]
colegi

In [26]:
df[df.isnull().any(axis=1)]

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU,lat,lng
8,9,Particular,Angol,San José,19,683.7,NaN,NaN
22,23,Particular,Talca,Particular Montessori,53,669.8,NaN,NaN
27,28,Particular,Las Condes,The Southern Cross School,31,668.6,NaN,NaN
44,45,Particular,Curicó,Alianza Francesa J. Mermoz,28,660.1,NaN,NaN
81,82,Particular,La Reina,Colegio British Royal School,63,648.0,NaN,NaN


In [27]:
df=df.dropna()
df.head(25)

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU,lat,lng
0,1,Particular,Providencia,Cambridge College,29,704.4,-33.429730,-70.617110
1,2,Particular,Las Condes,Cordillera,63,699.0,-33.397130,-70.514321
2,3,Particular,Concón,Montemar,22,694.0,-32.951048,-71.535463
3,4,Particular,Maipú,Internacional Alba,26,693.4,-33.480574,-70.744990
4,5,Particular,Vitacura,Los Andes,69,692.9,-33.375795,-70.527009
5,6,Particular,Puerto Montt,Instituto Alemán,46,690.9,-41.470315,-72.932780
6,7,Particular,Lo Barnechea,Maimonides School,10,689.7,-33.362362,-70.485016
7,8,Particular,Chiguayante,Pinares,16,685.5,-36.932918,-73.024080
9,10,Particular,La Reina,The Grange School,133,682.6,-33.436938,-70.567836
10,11,Particular,Lo Barnechea,Instituto Hebreo,76,681.0,-33.367826,-70.508976


In [28]:
map = folium.Map(location=[-33.39, -70.57], zoom_start=8, tiles='Stamen Terrain')
i=0
for index, row in df.iterrows():
    map.add_child(
        folium.CircleMarker([row.lat,row.lng],
                            color='#000000',
                            fill_color='#ff0000',
                            fill=True,
                            fill_opacity=0.7,
                            radius=5, 
                            popup=u"%s %s %s" % (row.Colegio, row['Promedio PSU'], row['Comuna'])))
map.save('colegios.html')
map

In [27]:
df.head()

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU,lat,lng
0,1,Particular,Providencia,Cambridge College,29,704.4,-33.429787,-70.616853
1,2,Particular,Las Condes,Cordillera,63,699.0,-33.396461,-70.513284
2,3,Particular,Concón,Montemar,22,694.0,-32.950793,-71.535073
3,4,Particular,Maipú,Internacional Alba,26,693.4,-33.480762,-70.745021
4,5,Particular,Vitacura,Los Andes,69,692.9,-33.375544,-70.526525


In [44]:
map = folium.Map(location=[-33.39, -70.57], zoom_start=8, tiles='Stamen Toner')
i=0
for index, row in df2.iterrows():
    map.add_child(
        folium.CircleMarker([row.lat,row.lng],
                            color='#ff0000',
                            fill_color='#ff0000',
                            fill=True,
                            fill_opacity=0.7,
                            radius=5*row['Alumnos que rindieron la prueba']/100.0, 
                            popup=u"%s %s %s" % (row.Colegio, row['Promedio PSU'], row['Comuna'])))
map.save('colegios.html')
map

In [28]:
df.head()

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU,lat,lng
0,1,Particular,Providencia,Cambridge College,29,704.4,-33.429787,-70.616853
1,2,Particular,Las Condes,Cordillera,63,699.0,-33.396461,-70.513284
2,3,Particular,Concón,Montemar,22,694.0,-32.950793,-71.535073
3,4,Particular,Maipú,Internacional Alba,26,693.4,-33.480762,-70.745021
4,5,Particular,Vitacura,Los Andes,69,692.9,-33.375544,-70.526525


In [13]:
df3 = df[df['Tipo de establecimiento'] != 'Particular']
df3

,Posición en el ranking,Tipo de establecimiento,Comuna,Colegio,Alumnos que rindieron la prueba,Promedio PSU,lat,lng
19,20,Subvencionado,Providencia,Josefino Santísima Trinidad,27,673.5,-33.435800,-70.619132
33,34,Municipal,Ñuñoa,Liceo Augusto D´Halmar,60,665.0,-33.460248,-70.580760
64,65,Subvencionado,San Vicente,Colegio El Salvador,65,652.5,-34.439336,-71.085276
82,83,Subvencionado,Vitacura,Liceo María Luisa Bombal,35,648.0,-33.388045,-70.579640
86,87,Subvencionado,La Serena,Colegio Gerónimo Rendic,112,647.5,-29.904211,-71.243276
96,97,Subvencionado,Copiapó,Colegio San Francisco de la Selva,15,643.2,-27.387174,-70.305078
98,99,Municipal,La Unión,Escuela El Maitén,1,643.0,-40.307150,-73.087335


In [72]:
map = folium.Map(location=[-33.39, -70.57], zoom_start=8, tiles='OpenStreetMap')
colors = {
    'Particular': ('green', ''),
    'Subvencionado': ('red', 'star'),
    'Municipal': ('red', 'star-empty'),
}
for index, row in df.iterrows():
    color, icon = colors[row['Tipo de establecimiento']]
    folium.Marker(
        location=[row.lat,row.lng],
        popup=u"%s - %s - %s - %s" % (row['Tipo de establecimiento'], row.Colegio, row['Promedio PSU'], row['Comuna']),
        icon=folium.Icon(color=color, icon=icon)
    ).add_to(map)
map.save('colegios.html')
map

[mapa](colegios.html)